In [1]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks
import tensorflow.keras as keras
from urllib.request import urlopen
import numpy as np
import cv2
from matplotlib import pyplot as plt

from albumentations import (
    HorizontalFlip,
    Transpose, ShiftScaleRotate, Resize, RandomSizedCrop, RandomCrop, Rotate
)

def augment(aug, image):
    image = aug(image=image)['image']
    return image

class CIFAR10AugmentationAndResize(tf.keras.utils.Sequence):
    #Default size is 96 because it is one of the default train image size of mobilenetv2
    def __init__(self, x_set, y_set, batch_size, 
                 augmentations = True,
                 resize = False,
                 height = 32, width = 32,
                 pShiftScaleRotate = .2, pRandomSizedCrop = .2):
        self.resize = resize
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.pShiftScaleRotate = pShiftScaleRotate
        self.pRandomSizedCrop = pRandomSizedCrop      
        self.height = height
        self.width = width
        self.augmentations = augmentations

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        images = [imread(file_name) for file_name in batch_x]
        
        if(self.resize):
            resize = Resize(height=self.height, width=self.width)
        rotate = Rotate()
        #shiftScaleRotate = ShiftScaleRotate(p=self.pShiftScaleRotate)
        randomSizedCrop = RandomSizedCrop(min_max_height=(self.height - self.height * .25 ,self.height), height=self.height, width=self.width, p=self.pRandomSizedCrop)
        horizontalFlip = HorizontalFlip()
        
        for i in range(len(images)):    
            if(self.resize):
                images[i] = augment(resize ,images[i])
            
            if(self.augmentations):
                images[i] = augment(rotate ,images[i])
                images[i] = augment(horizontalFlip ,images[i])
                images[i] = augment(randomSizedCrop ,images[i])
        
        return np.array(images)/255, np.array(batch_y)

In [2]:
def architecture5():
    model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (1, 1), input_shape=(32, 32, 3)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(16, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(32, (3, 3)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Conv2D(64, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(32, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(64, (3, 3)),
                               tf.keras.layers.BatchNormalization(),   
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Conv2D(128, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(64, 1,padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(128, (3, 3)),
                               tf.keras.layers.BatchNormalization(),  
                               tf.keras.layers.MaxPooling2D(2,2),
                               tf.keras.layers.ReLU(),
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dense(128, activation='relu'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dropout(.2),
                               tf.keras.layers.Dense(64, activation='relu'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Dropout(.2),
                               tf.keras.layers.Dense(10, activation='softmax')
                             ])
    return model
    

In [3]:
class SaveModel(callbacks.Callback):
    def __init__(self, filename):
        super(SaveModel, self).__init__()
        self.bestloss = 99999
        self.filename = filename
        
    def on_epoch_end(self, epoch, logs={}):
        val_loss, val_acc = logs['val_loss'], logs['val_accuracy']
        
        if(epoch % 50 == 0):
            self.model.save("checkpoint{}_{}".format(epoch // 50,self.filename), overwrite=True)
        
        if (self.bestloss > val_loss) :
            self.bestloss = val_loss
            print("\nUpdating best model")
            self.model.save("{}".format(self.filename), overwrite=True)
        return

In [4]:
import os
location =  r"E:\Data\CIFAR10\Original\Train"
files = sorted(os.listdir(location), key=lambda x : int(x[:-5]))
train = [ r"{}\{}".format(location,i) for i in files]

location = r"E:\Data\CIFAR10\Original"
file = "Train_Label.csv"
train_labels = pd.read_csv(r"{}/{}".format(location, file),index_col=0).values

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [5]:
from sklearn.model_selection import KFold


batch_size = 32
history = []
kf = KFold(n_splits=5, shuffle=True)
i = 0
for train_, valid in kf.split(train, train_labels):
    train_x, valid_x = np.array(train)[train_],  np.array(train)[valid]
    train_y, valid_y = np.array(train_labels)[train_],  np.array(train_labels)[valid]
    
    train_seq = CIFAR10AugmentationAndResize(train_x, train_y, batch_size)
    validation_seq = CIFAR10AugmentationAndResize(valid_x, valid_y, batch_size, augmentations=False)
    
    model = architecture5()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    callback = [SaveModel("architecture5_{}.h5".format(i))]
    i = i + 1
    
    history.append(model.fit_generator(train_seq, 
                                       validation_data=validation_seq, 
                                       epochs=100, shuffle=True, callbacks=callback))

Epoch 1/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.8347 - accuracy: 0.3468
Updating best model
1250/1250 [==============================] - 34s 27ms/step - loss: 1.8344 - accuracy: 0.3469 - val_loss: 1.5890 - val_accuracy: 0.4426
Epoch 2/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.5704 - accuracy: 0.4356
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.5703 - accuracy: 0.4357 - val_loss: 1.2853 - val_accuracy: 0.5442
Epoch 3/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.4647 - accuracy: 0.4775
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.4644 - accuracy: 0.4776 - val_loss: 1.2443 - val_accuracy: 0.5601
Epoch 4/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.3931 - accuracy: 0.5083
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.3932 - accuracy: 0.5082 - val_loss: 1.1794 - val_a

1250/1250 [==============================] - 27s 21ms/step - loss: 0.9249 - accuracy: 0.6827 - val_loss: 0.7486 - val_accuracy: 0.7423
Epoch 42/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.9168 - accuracy: 0.6849
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9168 - accuracy: 0.6848 - val_loss: 0.7231 - val_accuracy: 0.7510
Epoch 43/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9167 - accuracy: 0.6876 - val_loss: 0.7252 - val_accuracy: 0.7528
Epoch 44/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.9116 - accuracy: 0.6881
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9115 - accuracy: 0.6881 - val_loss: 0.7178 - val_accuracy: 0.7544
Epoch 45/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.9077 - accuracy: 0.6902
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9078 - accuracy: 0

Epoch 88/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8098 - accuracy: 0.7236 - val_loss: 0.6904 - val_accuracy: 0.7635
Epoch 89/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8078 - accuracy: 0.7242 - val_loss: 0.6775 - val_accuracy: 0.7699
Epoch 90/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.8111 - accuracy: 0.7213
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8113 - accuracy: 0.7212 - val_loss: 0.6617 - val_accuracy: 0.7742
Epoch 91/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8171 - accuracy: 0.7232 - val_loss: 0.6833 - val_accuracy: 0.7711
Epoch 92/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8043 - accuracy: 0.7266 - val_loss: 0.6960 - val_accuracy: 0.7675
Epoch 93/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8077 - accuracy: 0.7237 - val_loss: 0.6661 - val_accuracy: 0.7726
Epoch

1249/1250 [============================>.] - ETA: 0s - loss: 0.9667 - accuracy: 0.6684
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9667 - accuracy: 0.6683 - val_loss: 0.7512 - val_accuracy: 0.7368
Epoch 31/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9639 - accuracy: 0.6679 - val_loss: 0.7794 - val_accuracy: 0.7305
Epoch 32/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9621 - accuracy: 0.6679 - val_loss: 0.7530 - val_accuracy: 0.7378
Epoch 33/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9578 - accuracy: 0.6678 - val_loss: 0.7699 - val_accuracy: 0.7299
Epoch 34/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9542 - accuracy: 0.6722 - val_loss: 0.7842 - val_accuracy: 0.7287
Epoch 35/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9494 - accuracy: 0.6742 - val_loss: 0.8002 - val_accuracy: 0.7257
Epoch 36/100
1250/

1250/1250 [==============================] - 27s 22ms/step - loss: 0.8302 - accuracy: 0.7147 - val_loss: 0.7339 - val_accuracy: 0.7436
Epoch 78/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8379 - accuracy: 0.7147 - val_loss: 0.6870 - val_accuracy: 0.7686
Epoch 79/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8352 - accuracy: 0.7132 - val_loss: 0.6937 - val_accuracy: 0.7631
Epoch 80/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8292 - accuracy: 0.7175 - val_loss: 0.7113 - val_accuracy: 0.7596
Epoch 81/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8300 - accuracy: 0.7178 - val_loss: 0.7100 - val_accuracy: 0.7598
Epoch 82/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.8302 - accuracy: 0.7158
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8301 - accuracy: 0.7158 - val_loss: 0.6811 - val_accuracy: 0.7659
Epoch 83/100
1250/

...

Epoch 10/100
1250/1250 [==============================] - 27s 21ms/step - loss: 1.1832 - accuracy: 0.5882 - val_loss: 0.9666 - val_accuracy: 0.6648
Epoch 11/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.1681 - accuracy: 0.5964
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.1681 - accuracy: 0.5963 - val_loss: 0.9225 - val_accuracy: 0.6754
Epoch 12/100
1248/1250 [============================>.] - ETA: 0s - loss: 1.1464 - accuracy: 0.6027
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 1.1464 - accuracy: 0.6027 - val_loss: 0.8864 - val_accuracy: 0.6926
Epoch 13/100
1250/1250 [==============================] - 27s 21ms/step - loss: 1.1353 - accuracy: 0.6090 - val_loss: 0.9131 - val_accuracy: 0.6889
Epoch 14/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.1236 - accuracy: 0.6123
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.1239 

1250/1250 [==============================] - 27s 21ms/step - loss: 0.9075 - accuracy: 0.6856 - val_loss: 0.7087 - val_accuracy: 0.7585
Epoch 54/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.9030 - accuracy: 0.6920
Updating best model
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9029 - accuracy: 0.6920 - val_loss: 0.6848 - val_accuracy: 0.7633
Epoch 55/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8943 - accuracy: 0.6952 - val_loss: 0.7187 - val_accuracy: 0.7495
Epoch 56/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8936 - accuracy: 0.6916 - val_loss: 0.7007 - val_accuracy: 0.7577
Epoch 57/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8960 - accuracy: 0.6913 - val_loss: 0.7030 - val_accuracy: 0.7576
Epoch 58/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8845 - accuracy: 0.6952 - val_loss: 0.7032 - val_accuracy: 0.7566
Epoch 59/100
1250/

1249/1250 [============================>.] - ETA: 0s - loss: 1.8435 - accuracy: 0.3461
Updating best model
1250/1250 [==============================] - 40s 32ms/step - loss: 1.8436 - accuracy: 0.3461 - val_loss: 1.4540 - val_accuracy: 0.4739
Epoch 2/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.5615 - accuracy: 0.4438
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.5613 - accuracy: 0.4438 - val_loss: 1.3329 - val_accuracy: 0.5200
Epoch 3/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.4511 - accuracy: 0.4863
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 1.4511 - accuracy: 0.4863 - val_loss: 1.1707 - val_accuracy: 0.5796
Epoch 4/100
1250/1250 [==============================] - 27s 22ms/step - loss: 1.3956 - accuracy: 0.5079 - val_loss: 1.2528 - val_accuracy: 0.5610
Epoch 5/100
1249/1250 [============================>.] - ETA: 0s - loss: 1.3499 - accuracy: 0.5258
Upda

1250/1250 [==============================] - 27s 22ms/step - loss: 0.9155 - accuracy: 0.6871 - val_loss: 0.7298 - val_accuracy: 0.7485
Epoch 45/100
1249/1250 [============================>.] - ETA: 0s - loss: 0.9164 - accuracy: 0.6868
Updating best model
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9163 - accuracy: 0.6869 - val_loss: 0.6787 - val_accuracy: 0.7696
Epoch 46/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.9068 - accuracy: 0.6901 - val_loss: 0.6813 - val_accuracy: 0.7659
Epoch 47/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9123 - accuracy: 0.6877 - val_loss: 0.6961 - val_accuracy: 0.7580
Epoch 48/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9106 - accuracy: 0.6878 - val_loss: 0.6836 - val_accuracy: 0.7682
Epoch 49/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.9079 - accuracy: 0.6892 - val_loss: 0.6928 - val_accuracy: 0.7653
Epoch 50/100
1250/

1250/1250 [==============================] - 27s 21ms/step - loss: 0.8185 - accuracy: 0.7203 - val_loss: 0.6563 - val_accuracy: 0.7752
Epoch 95/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8200 - accuracy: 0.7204 - val_loss: 0.6718 - val_accuracy: 0.7713
Epoch 96/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8183 - accuracy: 0.7211 - val_loss: 0.6432 - val_accuracy: 0.7815
Epoch 97/100
1250/1250 [==============================] - 27s 21ms/step - loss: 0.8130 - accuracy: 0.7204 - val_loss: 0.6512 - val_accuracy: 0.7805
Epoch 98/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8096 - accuracy: 0.7244 - val_loss: 0.6506 - val_accuracy: 0.7785
Epoch 99/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8143 - accuracy: 0.7213 - val_loss: 0.6580 - val_accuracy: 0.7721
Epoch 100/100
1250/1250 [==============================] - 27s 22ms/step - loss: 0.8107 - accuracy: 0.7230 - val_loss: 0.6517

In [6]:
import os
location =  r"E:\Data\CIFAR10\Original\Test"
files = sorted(os.listdir(location), key=lambda x : int(x[:-5]))
test = [ r"{}\{}".format(location,i) for i in files]

location = r"E:\Data\CIFAR10\Original"
file = "Test_Label.csv"
test_labels = pd.read_csv(r"{}/{}".format(location, file),index_col=0).values

from sklearn.model_selection import train_test_split
import random

batch_size = 32
test_seq = CIFAR10AugmentationAndResize(test, test_labels, batch_size,height=96,augmentations=False)

In [7]:
results = []

for i in range(5):
    model = architecture5()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.load_weights("architecture5_{}.h5".format(i))
    results.append(model.evaluate_generator(test_seq))

In [12]:
loss, acc = 0, 0
for i, j in results:
    loss += i
    acc += j
loss/5, acc/5

(0.665889515415929, 0.772320008277893)